 # Extacción de datos

In [1]:
import pandas as pd
import fitz
import re

**Video:** https://drive.google.com/file/d/1M4_rOjzTI0hZz1FeO1vMEIxoBiVUoyxP/view

**Files:** 

Columnas(todas las columnas deben estar traducidas al español)
1. **IDCIS** (Número del control)
    - Clasificar entre nivel L1 y L2a
2. **Parametro** (Titulo)
    - Quitar lo que esta entre parentesis al final
3. **Descripcion**(Description)
    - Tomar la descripcion hasta el primer punto
    - Revisar los casos que tengan informacion importante
4. **Procedimiento de implementacion:** Remediation
    - The recommended state for this setting is: ... from  Description
    - revisar que no se traduzca la ruta //
5. **Procedimiento de verificacion:** Audit
    - Cuando audit no tenga recuadro amarillo colocar el de Remediation

## Extraer todos los controles

In [2]:
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0-1'
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0'
FILE='CIS_Microsoft_Windows_Server_2008_(non-R2)_Benchmark_v3.3.1_ARCHIVE'
#FILE='CIS_Microsoft_Windows_Server_2012_(non-R2)_Benchmark_v3.0.0_FINAL_UPDATE'
#FILE='CIS_Microsoft_Windows_Server_2016_Benchmark_v2.0.0'
#FILE='CIS_Microsoft_Windows_Server_2016_STIG_Benchmark_v2.0.0'

Se requiere extraer todos los controles del archivo pdf, para esto se utiliza la libreria `fitz` que permite extraer el texto de un archivo pdf, posterirmente se realiza scraping a bajo nivel mediante expresiones regulares para extraer los datos de cada control.

In [3]:
doc=fitz.open("../../source/Windows/"+FILE+".pdf")
doc.page_count

837

In [4]:
BEGIN_PAGE=0
END_PAGE=doc.page_count
allText=""
for i in range(BEGIN_PAGE,END_PAGE):
    page=doc.load_page(i)
    allText+=page.get_text("")
allText

' \n \nARCHIVE - CIS Microsoft \nWindows Server 2008 \n(non-R2) Benchmark \nv3.3.1 - 07-06-2023 \n \n \n \nPage 1 \nTerms of Use \nPlease see the below link for our current terms of use: \nhttps://www.cisecurity.org/cis-securesuite/cis-securesuite-membership-terms-of-use/ \n \n \n \nPage 2 \nTable of Contents \nTerms of Use ................................................................................................................. 1 \nTable of Contents .......................................................................................................... 2 \nOverview ...................................................................................................................... 20 \nIntended Audience ................................................................................................................20 \nConsensus Guidance ...........................................................................................................21 \nTypographical Conventions ..

Primero se debe elimar el indice y el apendice del documento actual debido a que contiene cadenas similares a las secciones que se desean extraer, el contenido que es interesante esta entre las secciones de Overview y Appendix de los documentos, por lo que se debe eliminar todo lo que este fuera de estas secciones.

In [5]:
PATTERN_SUBSTRACT_INDEX=re.compile(r"\nOverview \n(.*)\nAppendix: Summary Table", re.DOTALL)
allText=PATTERN_SUBSTRACT_INDEX.findall(allText)[0]
allText

'This is the final release of the CIS Benchmark for Microsoft Windows Server 2008 \n(non-R2) Benchmark. CIS encourages you to migrate to a more recent, supported \nversion of this technology. \nAll CIS Benchmarks focus on technical configuration settings used to maintain and/or \nincrease the security of the addressed technology, and they should be used in \nconjunction with other essential cyber hygiene tasks like: \n• Monitoring the base operating system for vulnerabilities and quickly updating with \nthe latest security patches  \n• Monitoring applications and libraries for vulnerabilities and quickly updating with \nthe latest security patches \nIn the end, the CIS Benchmarks are designed as a key component of a comprehensive \ncybersecurity program.  \nThis document provides prescriptive guidance for establishing a secure configuration \nposture for Microsoft Windows. \nThis secure configuration guide is based on Microsoft Windows Server 2008 and is \nintended for all versions of 

Ahora que se esta seguro de tener el texto que se desea, se procede a extraer cada uno de los controles los cuales se devuelven en un arreglo donde cada elemento es una cadena que representa un control y sus subsiones.

In [6]:
PATTERN_SECTIONS = re.compile(r'(\n\d+\.[\.\d]+ \(\w{2}\).*?)(?=\n\d+\.[\.\d]+ \(\w{2}\)|$)', re.DOTALL)
sections=PATTERN_SECTIONS.findall(allText)

In [7]:
sections

["\n1.1.1 (L1) Ensure 'Enforce password history' is set to '24 or more \npassword(s)' (Automated) \nProfile Applicability: \n•  Level 1 - Domain Controller \n•  Level 1 - Member Server \nDescription: \nThis policy setting determines the number of renewed, unique passwords that have to \nbe associated with a user account before you can reuse an old password. The value for \nthis policy setting must be between 0 and 24 passwords. The default value for stand-\nalone systems is 0 passwords, but the default setting when joined to a domain is 24 \npasswords. To maintain the effectiveness of this policy setting, use the Minimum \npassword age setting to prevent users from repeatedly changing their password. \nThe recommended state for this setting is: 24 or more password(s). \nNote: Password Policy settings (section 1.1) and Account Lockout Policy settings \n(section 1.2) must be applied via the Default Domain Policy GPO in order to be \nglobally in effect on domain user accounts as their def

## Extracción de secciones para cada control

El proceso conseiste en extraer cada una de las secciones requeridas para cada control, es necesario extraer las siguientes secciones:


1. **IDCIS**: Es el indice en el que aparece el control en el documento.
2. **Level**: Puede ser L1 o L2 y también se incluyen las clasificaciones 
3. **Description**: on a scale of 0 to 100, 0 being the lowest and 100 being the highest
4. **Audit**: different levels of education (e.g. "College", "PhD", "Masters", "Highschool")
5. **Remediation**: years of seniority


In [8]:

class Row:
    """
    Esta clase representa una fila o control con la información de un control
    """
    def __init__(self, IDCIS, Level, Title, Description, Audit, Remediation):
        self.IDCIS:str = IDCIS
        self.Level:str = Level
        self.Title:str = Title
        self.Description:str = Description
        self.Audit:str = Audit
        self.Remediation:str= Remediation
    
    def to_dict(self):
        return {"IDCIS":self.IDCIS, "Level":self.Level, "Title":self.Title, "Description":self.Description, "Audit":self.Audit, "Remediation":self.Remediation}


In [9]:
#WORKING
patternTitle = re.compile(r'[\.\d]+ \(\w{2}\)(.*?)Profile Applicability:', re.DOTALL)
patternDescription = re.compile(r'Description: \n(.*?)\nRationale:', re.DOTALL)
patternAudit = re.compile(r'\nAudit: \n(.*?)\nRemediation:', re.DOTALL)
patternRemediation = re.compile(r'Remediation: \n(.*?)\n(Default Value|CIS Controls)', re.DOTALL)
def extract_row(text):
    IDCS=re.search(r"[\.\d]+", text).group(0)+""
    Level=re.search(r"[\.\d]+ (\(\w{2}\))", text).group(1)+""
    Title=patternTitle.search(text).group(1)
    Description=patternDescription.search(text).group(1)
    Audit=patternAudit.search(text).group(1)
    Remediation=patternRemediation.search(text).group(1)
    
    return IDCS, Level, Title, Description, Audit, Remediation

In [10]:
df=pd.DataFrame(columns=["IDCIS", "Level", "Title", "Description", "Audit", "Remediation"])

In [11]:
i=0
for section in sections:
    row=Row(*extract_row(section))
    df.loc[i]=row.to_dict()
    i+=1
df

,IDCIS,Level,Title,Description,Audit,Remediation
0,1.1.1,(L1),Ensure 'Enforce password history' is set to '...,This policy setting determines the number of r...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
1,1.1.2,(L1),Ensure 'Maximum password age' is set to '365 ...,This policy setting defines how long a user ca...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
2,1.1.3,(L1),Ensure 'Minimum password age' is set to '1 or...,This policy setting determines the number of d...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
3,1.1.4,(L1),Ensure 'Minimum password length' is set to '1...,This policy setting determines the least numbe...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
4,1.1.5,(L1),Ensure 'Password must meet complexity require...,This policy setting checks all new passwords t...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
...,...,...,...,...,...,...
312,19.7.4.1,(L1),Ensure 'Do not preserve zone information in f...,This policy setting allows you to manage wheth...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
313,19.7.4.2,(L1),Ensure 'Notify antivirus programs when openin...,This policy setting manages the behavior for n...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
314,19.7.28.1,(L1),Ensure 'Prevent users from sharing files with...,This policy setting determines whether users c...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
315,19.7.43.1,(L1),Ensure 'Always install with elevated privileg...,This setting controls whether or not Windows I...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...


In [12]:
df.to_excel("../../output/Windows/extracted/"+FILE+"_extracted"+".xlsx", index=False)